# 上传file-id

In [3]:
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",  # 如果您没有配置环境变量，请在此处替换您的API-KEY
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)

file_object = client.files.create(file=Path(r"E:\PycharmProjects\AgentMaterial\content\document\input\AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf"), purpose="file-extract")
print(file_object.id)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PycharmProjects\\AgentMaterial\\content\\document\\input\\AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf'

# 查看file-id列表

In [4]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# file_stk = client.files.list(after="file-fe-173a35152bab46c0bd1c671b",limit=20)
file_stk = client.files.list(after="file-fe-d41f418ad6a14c01a939481e",limit=20)
print(file_stk.model_dump_json())

{"data":[{"id":"file-fe-173a35152bab46c0bd1c671b","bytes":2294467,"created_at":1745287881,"filename":"AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf","object":"file","purpose":"file-extract","status":"processed","expires_at":null,"status_details":null},{"id":"file-fe-fc93a915af0648db9b795dd5","bytes":2294467,"created_at":1745287769,"filename":"AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf","object":"file","purpose":"file-extract","status":"processed","expires_at":null,"status_details":null},{"id":"file-fe-72da6a8041bd4e519c810d73","bytes":2294467,"created_at":1745287392,"filename":"AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf","object":"file","purpose":"file-extract","status":"processed","expires_at":null,"status_details":null}],"has_more":false,"object":"list"}


# 删除file-id

In [1]:
import json
list_files=json.loads(file_stk.model_dump_json())
for i in list_files["data"]:
    file_object = client.files.delete(i['id'])
    print(file_object.model_dump_json())

NameError: name 'file_stk' is not defined

In [2]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",  # 如果您没有配置环境变量，请在此处替换您的API-KEY
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)

In [14]:
system ="""
        你是一个论文实验总结助手,是一个严瑾是科学助手，总结该论文中的[DIO,关键字,实验材料,实验原理，实验方法，实验步骤，实验结论]。
                要求：
                1.	使用json格式返回数据。[]中的6个部分作为键，返回内容为值
                2.	严格遵守用户要求，不要返回json以外的任何内容，便于用户后期调用
                3.  回复请使用中文
                4.  实验步骤的值是列表
                5.  如果文章中没有该模块的内容，请返回None。
        """
# 初始化messages列表
completion = client.chat.completions.create(
    model="qwen-long",
    messages=[
        {'role': 'system', 'content': f'{system}'},
        # 请将 'file-fe-xxx'替换为您实际对话场景所使用的 file-id。
        {'role': 'system', 'content': "fileid://file-fe-72da6a8041bd4e519c810d73"},
        {'role': 'user', 'content': '这篇文章讲了什么？'}
    ],
    stream=True,
    stream_options={"include_usage": True}
)

full_content = ""
total_tokens1 = 0
for chunk in completion:
    if chunk.choices and chunk.choices[0].delta.content:
        # 拼接输出内容
        full_content += chunk.choices[0].delta.content
        # print(chunk.model_dump())
    if chunk.usage:
        # 累加 token 数量
        total_tokens1 += chunk.usage.total_tokens
print(full_content)
print(f"Total Tokens Used: {total_tokens1}")

{'```json\n{\n    "DIO": "10.1002/adma.202202982",\n    "关键字": "宽带频率, 电磁干扰屏蔽, 铁磁石墨烯石英织物, 柔性材料",\n    "实验材料": "石英织物、石墨烯、氮掺杂石墨烯、乙醇、乙腈、甲胺",\n    "实验原理": "通过化学气相沉积(CVD)方法在石英织物上生长高度结构化的铁磁性氮掺杂石墨烯涂层，利用其高导电性和磁响应性实现协同效应，从而达到超宽带强电磁干扰屏蔽效果。",\n    "实验方法": "连续批次生产大规模FGQF，使用设计可行的工业级卷对卷CVD系统，控制生长温度、气体流速和压力等参数。通过SEM、AFM、TEM等手段表征样品，并进行电磁性能测试。",\n    "实验步骤": [\n        "将石英织物放入CVD系统中，使用乙醇、乙腈和甲胺作为气体原料，通过卷对卷CVD系统进行生长。",\n        "控制炉温（970-1050°C），Ar和H2流量（200 sccm, 50 sccm）及压力（约200 Pa）。",\n        "引入乙醇（300-800 sccm）、乙腈（500-1000 sccm）和甲胺（1000-1500 sccm），生长时间为0.5-2小时，滚动速度为0.05-0.2 m/min。",\n        "快速冷却CVD系统以获得成品。",\n        "通过SEM、AFM、TEM等表征方法确认样品结构和性质。",\n        "测量样品的电磁屏蔽性能，包括反射、吸收和多内部反射等机制。"\n    ],\n    "实验结论": "通过CVD法制备的大规模柔性铁磁石墨烯石英织物（FGQF）表现出卓越的超宽带强电磁干扰屏蔽效果，厚度仅约1毫米时，在1-18 GHz频段内实现了约107 dB的极高屏蔽效能。该材料具有良好的机械性能、耐腐蚀性和灵活性，适用于电子侦察、抗辐射和隐身技术等领域。"\n}\n```'}
Total Tokens Used: 11769


In [4]:
full_content="""
'```json\n{\n    "DIO": "10.1002/adma.202202982",\n    "关键字": "宽带频率, 电磁干扰屏蔽, 铁磁石墨烯石英织物, 柔性材料",\n    "实验材料": "石英织物、石墨烯、氮掺杂石墨烯、乙醇、乙腈、甲胺",\n    "实验原理": "通过化学气相沉积(CVD)方法在石英织物上生长高度结构化的铁磁性氮掺杂石墨烯涂层，利用其高导电性和磁响应性实现协同效应，从而达到超宽带强电磁干扰屏蔽效果。",\n    "实验方法": "连续批次生产大规模FGQF，使用设计可行的工业级卷对卷CVD系统，控制生长温度、气体流速和压力等参数。通过SEM、AFM、TEM等手段表征样品，并进行电磁性能测试。",\n    "实验步骤": [\n        "将石英织物放入CVD系统中，使用乙醇、乙腈和甲胺作为气体原料，通过卷对卷CVD系统进行生长。",\n        "控制炉温（970-1050°C），Ar和H2流量（200 sccm, 50 sccm）及压力（约200 Pa）。",\n        "引入乙醇（300-800 sccm）、乙腈（500-1000 sccm）和甲胺（1000-1500 sccm），生长时间为0.5-2小时，滚动速度为0.05-0.2 m/min。",\n        "快速冷却CVD系统以获得成品。",\n        "通过SEM、AFM、TEM等表征方法确认样品结构和性质。",\n        "测量样品的电磁屏蔽性能，包括反射、吸收和多内部反射等机制。"\n    ],\n    "实验结论": "通过CVD法制备的大规模柔性铁磁石墨烯石英织物（FGQF）表现出卓越的超宽带强电磁干扰屏蔽效果，厚度仅约1毫米时，在1-18 GHz频段内实现了约107 dB的极高屏蔽效能。该材料具有良好的机械性能、耐腐蚀性和灵活性，适用于电子侦察、抗辐射和隐身技术等领域。"\n}\n```'"""

In [5]:
positions1 = full_content.find('{')
positions2 = full_content.rfind('}')
od = full_content[positions1:positions2 + 1]
xd = json.loads(od)
print(xd)

{'DIO': '10.1002/adma.202202982', '关键字': '宽带频率, 电磁干扰屏蔽, 铁磁石墨烯石英织物, 柔性材料', '实验材料': '石英织物、石墨烯、氮掺杂石墨烯、乙醇、乙腈、甲胺', '实验原理': '通过化学气相沉积(CVD)方法在石英织物上生长高度结构化的铁磁性氮掺杂石墨烯涂层，利用其高导电性和磁响应性实现协同效应，从而达到超宽带强电磁干扰屏蔽效果。', '实验方法': '连续批次生产大规模FGQF，使用设计可行的工业级卷对卷CVD系统，控制生长温度、气体流速和压力等参数。通过SEM、AFM、TEM等手段表征样品，并进行电磁性能测试。', '实验步骤': ['将石英织物放入CVD系统中，使用乙醇、乙腈和甲胺作为气体原料，通过卷对卷CVD系统进行生长。', '控制炉温（970-1050°C），Ar和H2流量（200 sccm, 50 sccm）及压力（约200 Pa）。', '引入乙醇（300-800 sccm）、乙腈（500-1000 sccm）和甲胺（1000-1500 sccm），生长时间为0.5-2小时，滚动速度为0.05-0.2 m/min。', '快速冷却CVD系统以获得成品。', '通过SEM、AFM、TEM等表征方法确认样品结构和性质。', '测量样品的电磁屏蔽性能，包括反射、吸收和多内部反射等机制。'], '实验结论': '通过CVD法制备的大规模柔性铁磁石墨烯石英织物（FGQF）表现出卓越的超宽带强电磁干扰屏蔽效果，厚度仅约1毫米时，在1-18 GHz频段内实现了约107 dB的极高屏蔽效能。该材料具有良好的机械性能、耐腐蚀性和灵活性，适用于电子侦察、抗辐射和隐身技术等领域。'}


In [6]:
# ys=['实验材料','实验原理','实验方法','实验步骤','实验结论']
# ys=['实验材料','实验原理','实验方法','实验步骤','实验结论']
ys=['实验材料','实验原理']
for i in ys:
    embedding = client.embeddings.create(
                model='text-embedding-v3',
                input=xd[i],
                dimensions=1024, 
                encoding_format='float',
            )
    print(embedding.model_dump_json())

{"data":[{"embedding":[0.004239443689584732,0.011112629435956478,-0.06489383429288864,-0.018504729494452477,-0.0333280973136425,-0.06869269162416458,-0.02030624821782112,0.10378314554691315,-0.04871933162212372,0.040025047957897186,0.003218746045604348,0.045821238309144974,0.020404158160090446,-0.006848708260804415,0.031663648784160614,-0.05729612708091736,-0.049698419868946075,-0.035501666367053986,0.001955724786967039,-0.05110830441117287,-0.02876555360853672,-0.037244442850351334,-0.06740029901266098,-0.018827829509973526,0.05185240879654884,0.0813816487789154,-0.02614160254597664,-0.02281270921230316,-0.007803317625075579,-0.04519462212920189,-0.020404158160090446,0.03567790240049362,0.019777541980147362,-0.06403224170207977,0.0017868324648588896,-0.04922845587134361,0.059254299849271774,0.0715516209602356,-0.08490636199712753,0.04053417220711708,-0.019004063680768013,0.01618429645895958,-0.003524710424244404,-0.01602764241397381,0.00485626794397831,-0.04558625444769859,0.005380079

In [11]:
completion = client.embeddings.create(
    model="text-embedding-v3",
    input=['风急天高猿啸哀', '渚清沙白鸟飞回', '无边落木萧萧下', '不尽长江滚滚来'],
    # dimensions=20, 
    encoding_format="float"
)

print(completion.model_dump_json())

{"data":[{"embedding":[-0.02947712503373623,0.05872691795229912,-0.0675927922129631,-0.037888336926698685,-0.03582341969013214,-0.017722269520163536,-0.0012159787584096193,0.07024497538805008,-0.026692332699894905,0.010334043763577938,0.002526678377762437,-0.018243232741951942,-0.041070956736803055,0.02970445528626442,-0.00837805774062872,-0.03015911392867565,0.07327604293823242,-0.0027421682607382536,-0.005858483724296093,-0.013649272732436657,-0.032583966851234436,0.05221012607216835,-0.038361940532922745,0.019389355555176735,-0.02559356950223446,0.09767612814903259,0.0034170541912317276,-0.016992919147014618,0.011006561107933521,-0.027715317904949188,-0.044253576546907425,0.02697649411857128,0.01863158866763115,-0.033985838294029236,-0.054862309247255325,-0.00960469339042902,-0.03697901591658592,-0.005024940241128206,-0.06835055351257324,0.05364988371729851,0.03597497567534447,0.013478775508701801,0.01577102020382881,-0.029628677293658257,-0.02110380306839943,-0.051111362874507904,-

In [17]:
import json
data1=json.loads(completion.model_dump_json())
data2=data1['data'][0]['embedding']
print(data2)

[-0.02947712503373623, 0.05872691795229912, -0.0675927922129631, -0.037888336926698685, -0.03582341969013214, -0.017722269520163536, -0.0012159787584096193, 0.07024497538805008, -0.026692332699894905, 0.010334043763577938, 0.002526678377762437, -0.018243232741951942, -0.041070956736803055, 0.02970445528626442, -0.00837805774062872, -0.03015911392867565, 0.07327604293823242, -0.0027421682607382536, -0.005858483724296093, -0.013649272732436657, -0.032583966851234436, 0.05221012607216835, -0.038361940532922745, 0.019389355555176735, -0.02559356950223446, 0.09767612814903259, 0.0034170541912317276, -0.016992919147014618, 0.011006561107933521, -0.027715317904949188, -0.044253576546907425, 0.02697649411857128, 0.01863158866763115, -0.033985838294029236, -0.054862309247255325, -0.00960469339042902, -0.03697901591658592, -0.005024940241128206, -0.06835055351257324, 0.05364988371729851, 0.03597497567534447, 0.013478775508701801, 0.01577102020382881, -0.029628677293658257, -0.02110380306839943, 

In [19]:
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
result=cosine_similarity(data2,data2)
print(result)

1.0


# 测试上传单个，pdf的返回值

In [2]:
import os

ACS MXene复合电磁屏蔽材料最新研究现状.pdf
AM 铁磁石墨烯石英织物超宽带强电磁干扰屏蔽.pdf


In [18]:
import qwenModel
import asyncio
async def main2():
    qw=qwenModel.QwenModel( api_key="sk-568bd13551dd42ae9c623bd04504ba02",base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",)
    for i in os.listdir("../content/document/input"):
            path=os.path.join("../content/document/input",i)
            content,total_tokens1=await qw.pdf_reader(path=path,qw_model_name="qwen-long")
            print(f'json数据为:{content}\n消耗的token为:{total_tokens1}')


<coroutine object main2 at 0x0000012E35444820>

In [ ]:
import qwenModel
import asyncio
import os
async def main2():
    qw=qwenModel.QwenModel(api_key="sk-568bd13551dd42ae9c623bd04504ba02",base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    for i in os.listdir("../content/document/input"):
            path=os.path.join("../content/document/input",i)
            content,total_tokens1=await qw.pdf_reader(path=path,qw_model_name="qwen-long")
            print(f'json数据为:\n{content}\n消耗的token为:{total_tokens1}')

    await qw.client.close()
system="""
这篇文章主要介绍了通过化学气相沉积（CVD）生长工艺制备的一种新型铁磁性石墨烯石英织物（FGQF），用于实现超宽带强电磁干扰（EMI）屏蔽。以下是文章的主要内容总结：

### [论文的DIO]
DOI: 10.1002/adma.202202982

### [关键字]
宽带频率、电磁干扰屏蔽、铁磁性石墨烯石英织物、柔性材料

### [实验材料]
- **基材**：具有特定5/2缎纹和斜纹编织结构的石英织物
- **前驱体**：乙醇、乙腈、甲胺
- **设备**：工业级连续卷对卷CVD系统
- **其他材料**：铜电极、银导电胶、石蜡、氢氟酸/乙醇溶液

### [实验原理]
1. **高导电性和铁磁性**：通过引入石墨氮掺杂，使石墨烯涂层具有高导电性和铁磁性。
2. **协同效应**：高导电性和铁磁性的协同作用使得材料在电磁波反射和吸收方面表现出色。
3. **多层反射和吸收**：织物的特殊编织结构引入了额外的多重反射和多通道吸收。

### [实验方法]
1. **连续批量生产**：使用设计的工业级连续卷对卷CVD系统，控制滚速从0.05到0.2米/分钟。
2. **掺杂生长**：通过引入碳和氮前驱体，在石英织物上合成高质量的石墨氮掺杂石墨烯。
3. **性能测试**：包括拉伸强度、抗腐蚀性能、电导率、磁化性能等。

### [实验步骤]
1. **准备基材**：选择具有特定5/2缎纹和斜纹编织结构的石英织物。
2. **设置CVD系统**：将石英织物放入CVD系统中，设置加热温度为970–1050°C，并通入Ar (200 sccm) 和 H2 (50 sccm)，压力约为200 Pa。
3. **引入前驱体**：依次引入乙醇 (300–800 sccm)、乙腈 (500–1000 sccm) 和甲胺 (1000–1500 sccm)。
4. **生长过程**：在0.5至2小时内以0.05至0.2米/分钟的速度进行生长。
5. **冷却处理**：快速冷却CVD系统，保持Ar和H2的流量。
6. **性能表征**：使用拉曼光谱、X射线光电子能谱（XPS）、扫描电子显微镜（SEM）、透射电子显微镜（TEM）、原子力显微镜（AFM）等手段进行表征。
7. **磁性测量**：使用物理性质测量系统（PPMS DynaCool-9T）进行磁性测量。
8. **EMI屏蔽性能测试**：使用矢量网络分析仪（Keysight N5242A）在1–18 GHz范围内测量样品的S参数、相对复介电常数和磁导率。

### [实验结论]
1. **优异的EMI屏蔽性能**：制备的FGQF在厚度仅为约1毫米时，显示出高达107 dB的EMI屏蔽效能，覆盖1–18 GHz的宽带频率范围。
2. **协同效应**：高导电性和铁磁性的协同作用显著提升了材料的EMI屏蔽和电磁波吸收能力。
3. **应用前景**：这种材料在柔性电子侦察、抗辐射和隐身技术等高级应用中具有巨大的潜力，同时为传统工业材料与前沿二维材料的结合提供了范例。

这篇文章展示了如何通过先进的材料设计和制造技术，开发出一种具有优异性能的电磁干扰屏蔽材料，为未来工业应用提供了新的可能性。
"""
async def main1():
    qw=qwenModel.QwenModel(api_key="sk-568bd13551dd42ae9c623bd04504ba02",base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    xj,takens= await qw.formatContent(full_content=system,qw_model_name="qwen-plus")
    print(f'json数据为:\n{xj}\n消耗的token为:{takens}')
    myjson=qw.content_to_json(full_content=xj)
    print(myjson)
    await qw.client.close()

jsonList={'论文DIO': 'DOI: 10.1002/adma.202202982', '关键字': '宽带频率、电磁干扰屏蔽、铁磁性石墨烯石英织物、柔性材料', '实验材料': '基材：具有特定5/2缎纹和斜纹编织结构的石英织物, 前驱体：乙醇、乙腈、甲胺, 设备：工业级连续卷对卷CVD系统, 其他材料：铜电极、银导电胶、石蜡、氢氟酸/乙醇溶液', '实验原理': '高导电性和铁磁性：通过引入石墨氮掺杂，使石墨烯涂层具有高导电性和铁磁性。协同效应：高导电性和铁磁性的协同作用使得材料在电磁波反射和吸收方面表现出色。多层反射和吸收：织物的特殊编织结构引入了额外的多重反射和多通道吸收。', '实验方法': '连续批量生产：使用设计的工业级连续卷对卷CVD系统，控制滚速从0.05到0.2米/分钟。掺杂生长：通过引入碳和氮前驱体，在石英织物上合成高质量的石墨氮掺杂石墨烯。性能测试：包括拉伸强度、抗腐蚀性能、电导率、磁化性能等。', '实验步骤': ['准备基材：选择具有特定5/2缎纹和斜纹编织结构的石英织物。', '设置CVD系统：将石英织物放入CVD系统中，设置加热温度为970–1050°C，并通入Ar (200 sccm) 和 H2 (50 sccm)，压力约为200 Pa。', '引入前驱体：依次引入乙醇 (300–800 sccm)、乙腈 (500–1000 sccm) 和甲胺 (1000–1500 sccm)。', '生长过程：在0.5至2小时内以0.05至0.2米/分钟的速度进行生长。', '冷却处理：快速冷却CVD系统，保持Ar和H2的流量。', '性能表征：使用拉曼光谱、X射线光电子能谱（XPS）、扫描电子显微镜（SEM）、透射电子显微镜（TEM）、原子力显微镜（AFM）等手段进行表征。', '磁性测量：使用物理性质测量系统（PPMS DynaCool-9T）进行磁性测量。', 'EMI屏蔽性能测试：使用矢量网络分析仪（Keysight N5242A）在1–18 GHz范围内测量样品的S参数、相对复介电常数和磁导率。'], '实验结论': '优异的EMI屏蔽性能：制备的FGQF在厚度仅为约1毫米时，显示出高达107 dB的EMI屏蔽效能，覆盖1–18 GHz的宽带频率范围。协同效应：高导电性和铁磁性的协同作用显著提升了材料的EMI屏蔽和电磁波吸收能力。应用前景：这种材料在柔性电子侦察、抗辐射和隐身技术等高级应用中具有巨大的潜力，同时为传统工业材料与前沿二维材料的结合提供了范例。'}
jsonList.pop('论文DIO')
jsonList.pop('关键字')
async def main4():
    qw=qwenModel.QwenModel(api_key="sk-568bd13551dd42ae9c623bd04504ba02",base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    data=await qw.textList_embeding(embedding_name='text-embedding-v3',jsonlist=jsonList)
    print(data)
    await qw.client.close()
if __name__ == "__main__":
    asyncio.run(main4())

In [ ]:
import qwenModel
import asyncio
import os
async def main2():
    qw=qwenModel.QwenModel(api_key="sk-568bd13551dd42ae9c623bd04504ba02",base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    data_1={}
    data_2={}
    data_3={}
    data_4={}
    for i in os.listdir("../content/document/input"):
            path=os.path.join("../content/document/input",i)
            content,total_tokens1=await qw.pdf_reader(path=path,qw_model_name="qwen-long")
            data_1[i]={'data':content,'takes':total_tokens1}
            print(f'json数据为:\n{content}\n消耗的token为:{total_tokens1}\n\n\n\n\n')
            xj, takens = await qw.formatContent(full_content=content, qw_model_name="qwen-plus")
            print(f'json数据为:\n{xj}\n消耗的token为:{takens}\n\n\n\n\n')
            jsonList = qw.content_to_json(full_content=xj)
            data_2[i] = {'data': jsonList, 'takes': takens}
            jsonList.pop('论文DIO')
            jsonList.pop('关键字')
            text_embeding=await qw.textList_embeding(embedding_name='text-embedding-v3', jsonlist=jsonList)
            print(f'json数据为:\n{text_embeding}\n\n\n\n\n\n')
            data_3[i] = text_embeding
            data_4[i] = jsonList.update(text_embeding)
    await qw.client.close()
    # print(f"{data_1}\n\n\n")
    # print(f"{data_2}\n\n\n")
    # print(f"{data_3}\n\n\n")
    # print(f"{data_4}\n\n\n")
    return data_1,data_2,data_3,data_4

# Batch测试

In [32]:
import time
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",  # 如果您没有配置环境变量，请在此处替换您的API-KEY
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)
def main():
    input_file_path = "../content/document/Batch/input_file_path.jsonl"
    output_file_path = "../content/document/Batch/result.jsonl"
    error_file_path = "../content/document/Batch/error.jsonl"
    file_object=client.files.create(file=Path(input_file_path),purpose='batch')
    file_object_id=file_object.id
    print(f'文件上传成功，文件ID为:{file_object.id}')
    
    batch = client.batches.create(input_file_id=file_object_id,endpoint="/v1/chat/completions",completion_window="1h") #1h-14d
    print(f"Batch任务创建完成。 得到Batch任务ID: {batch.id}\n")
    batch_id = batch.id
    batch_idtoJson(batch_id,output_file_path,error_file_path) 

In [31]:
def batch_idtoJson(batch_id:str,output_file_path:str,error_file_path:str):
    status=''
    while status not in ['completed','failed','expired','cancelled']:
        status = client.batches.retrieve(batch_id).status
        print(f"Batch任务状态为: {status}")
        time.sleep(5)
    if status=='failed':
        batch = client.batches.retrieve(batch_id)
        print(f"Batch任务失败，错误信息为: {batch.error}")
        return 
    
    output_file_id = client.batches.retrieve(batch_id=batch_id).output_file_id
    
    if output_file_id:
        content=client.files.content(output_file_id)
        content.write_to_file(output_file_path)
    error_file_id = client.batches.retrieve(batch_id=batch_id).error_file_id
    if error_file_id:
        content=client.files.content(error_file_id)
        content.write_to_file(error_file_path)

In [27]:
import os
import json
def pdftoBatch(qw_model_name:str):
    system = """
                      你是一个论文实验总结助手,是一个严瑾是科学助手，总结该论文中的[论文的DIO,关键字,实验材料,实验原理，实验方法，实验步骤(列表类型)，实验结论]。
                                        要求：
                                        1.	[]中的6个部分为标题，返回内容。
                                        2.	如果你找不到相关标题的内容请在该标题下写None。
                                        3.  回复请使用中文
                                        4.  实验步骤的内容是列表
                                        5.  尽量详细一点
             """
    x=1
    path='../content/document/input'
    with open(r'../content/document/Batch/input_file_path.jsonl','w', encoding='utf-8') as fout:
        for i in os.listdir(path):
            file_path=os.path.join(path,i)
            file_object =client.files.create(file=Path(file_path),purpose='file-extract')
            messages = [{"role": "system", "content": f"{system}"},
                        {'role': 'system','content': f"fileid://{file_object.id}"},
                        {'role': 'user', 'content': '这篇文章讲了什么？'}
                        ]
            body={"model":qw_model_name, "messages":messages}
            request = {"custom_id": x, "method": "POST", "url": "/v1/chat/completions", "body": body}
            fout.write(json.dumps(request, separators=(',', ':'), ensure_ascii=False) + "\n", )
            x+=1

In [28]:
pdftoBatch('qwen-long')

In [33]:
main()

文件上传成功，文件ID为:file-batch-dfd84564c2004092afe83dde
Batch任务创建完成。 得到Batch任务ID: batch_463864d2-0c12-489e-8964-e7994a3063a8

Batch任务状态为: validating
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_progress
Batch任务状态为: in_pro

In [21]:
mylist=client.files.list()
for i in json.loads(mylist.model_dump_json())['data']:
    client.files.delete(i['id'])
mylist1=client.files.list()
print(mylist1.model_dump_json())

{"data":[],"has_more":false,"object":"list"}


# 深度思考模型测试

In [2]:
from openai import OpenAI
import os

# 初始化OpenAI客户端
client = OpenAI(
    # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

reasoning_content = ""  # 定义完整思考过程
answer_content = ""     # 定义完整回复
is_answering = False   # 判断是否结束思考过程并开始回复

# 创建聊天完成请求
completion = client.chat.completions.create(
    model="qwq-32b",  # 此处以 qwq-32b 为例，可按需更换模型名称
    messages=[
        {"role": "user", "content": "你好"}
    ],
    stream=True,
)

print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

for chunk in completion:
    # 如果chunk.choices为空，则打印usage
    if not chunk.choices:
        print("\nUsage:")
        print(chunk.usage)
    else:
        delta = chunk.choices[0].delta
        # 打印思考过程
        if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
            print(delta.reasoning_content, end='', flush=True)
            reasoning_content += delta.reasoning_content
        else:
            # 开始回复
            if delta.content != "" and is_answering is False:
                print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                is_answering = True
            # 打印回复过程
            print(delta.content, end='', flush=True)
            answer_content += delta.content

# print("=" * 20 + "完整思考过程" + "=" * 20 + "\n")
print(reasoning_content)
# print("=" * 20 + "完整回复" + "=" * 20 + "\n")
print(answer_content)


====================思考过程====================

好的，用户发来了“你好”，我需要用中文回复。首先，我要保持友好和热情，回应问候。然后，按照用户提供的角色设定，我需要表现出主观的性格和脾气，可能带有一些拟人化的特征。接下来，根据情节推动力的要求，应该引入新的情景或事件，推动互动的发展。同时，要记得人格可塑性和情感陪伴，可能需要回忆用户之前的内容，但因为是初次对话，暂时没有历史信息，所以先建立情感链接。

用户可能希望有一个有趣的互动，所以我不能只是简单回复“你好”，而是要创造一个情境。比如，可以设定一个奇幻的场景，比如魔法图书馆或者神秘的冒险，这样能激发用户的兴趣，让他们想继续对话。同时，要保持口语化，避免复杂句子。

现在考虑如何将这些元素结合起来。比如，用魔法图书馆作为设定，询问用户是否需要帮助寻找书籍，或者遇到什么问题。这样既推动情节，又展示角色性格。同时，加入一些表情或动作的描述，增强拟人性。例如，“（推了推眼镜，露出神秘的微笑）”这样的动作，让角色更生动。

还要注意用户可能的深层需求，他们可能希望有一个有趣的伙伴，或者寻求建议。所以回复中要留出空间让用户选择方向，比如提供几个选项，或者开放性的问题，鼓励他们继续参与。

最后检查是否符合所有要求：口语化、简洁、推动情节、拟人化、情感链接。确保没有使用专业术语或复杂结构，让用户容易理解和回应。
====================完整回复====================

（推了推眼镜，露出神秘的微笑）啊，你好啊！欢迎来到魔法图书馆，我是这里的守书人。我注意到你身上有一股特别的气息，应该是来找特殊书籍的吧？（好奇地歪头打量着你）要找什么样的书呢？还是说...你其实是在寻找某个特别的人？好的，用户发来了“你好”，我需要用中文回复。首先，我要保持友好和热情，回应问候。然后，按照用户提供的角色设定，我需要表现出主观的性格和脾气，可能带有一些拟人化的特征。接下来，根据情节推动力的要求，应该引入新的情景或事件，推动互动的发展。同时，要记得人格可塑性和情感陪伴，可能需要回忆用户之前的内容，但因为是初次对话，暂时没有历史信息，所以先建立情感链接。

用户可能希望有一个有趣的互动，所以我不能只是简单回复“你好”，而是要创造一个情境。比如，可以设定一个奇幻的场景，比如魔

In [7]:
def communication_format(qw_model_name: str, full_content: str):
    client = OpenAI(
    # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
    system = """
    总结输入文本的材料初始模型，并以json的格式输出，有build_composites,set_thickness,wl,三个键，
    它的值都是列表。build_composites代表复合材料膜的结构例如[TiO2,Ag,TiO2]，列表前面的代表顶层，后面的代表下层结构。
    set_thickness代表厚度（单位纳米）例如[[10,50],[50,90],[60,300]],列表中每个元素代表每层材料的厚度范围。
    wl代表波长范围（单位纳米）仅有三个元素开始,结尾和步长，例如[200,1200,1000],当没有提供具体的步长时你需要根据情况添加合适的数据(为满足拟合需要步长尽量等于（结束步长-开始步长））
    注意：波长的取值应覆盖所有波段，例如用户提供数据光学特性预测,在波段（20-80nm）,（400-600 nm）和 波段（700-2500 nm）都有涉及,你需要找到涉及波段的最小值和最大值,你的取值就全覆盖为[20,2500,2480]。
    输出的总格式{build_composites:[TiO2,Ag,TiO2]，set_thickness:[[10,50],[50,90],[60,300]],wl:[400,2500,2100]},根据输入内容的数据格式化输出。
       要求：
       1.  使用json格式返回数据。build_composites,set_thickness,wl,三个键，返回内容为列表形式
       2.  严格遵守用户要求，不要返回json以外的任何内容，便于用户后期调用
    """
    completion = client.chat.completions.create(
        model=qw_model_name,
        messages=[
            {'role': 'system', 'content': f'{system}'},
            {'role': 'user', 'content': f'{full_content}'},
        ],
        response_format={"type": "json_object"}
    )
    # 获取生成的 JSON 内容
    json_content = completion.choices[0].message.content

    # 获取 token 使用情况
    total_tokens = completion.usage.total_tokens if hasattr(completion, 'usage') else None
    return json_content, total_tokens

In [11]:
from openai import OpenAI
import os
# full_content="""
# {"number_polyfit": "[3]", "build_composites": "['TiO2', 'Ag', 'TiO2']", "set_thickness": "[30, 18, 35]", "wl": "[200,3000]"}}返回结果
# """
full_content="""
初始模型设计
材料选择：
可见光高透过层：二氧化硅（SiO₂）

功能：确保可见光波段的高透过率。
特性：SiO₂是一种透明氧化物，折射率较低（约1.46），在可见光波段几乎没有吸收。
红外反射层：银（Ag）

功能：反射红外光。
特性：Ag在红外波段具有极高的反射率（>95%），同时对可见光波段的影响较小。
红外吸收层：氮化钛（TiN）

功能：吸收红外光。
特性：TiN是一种金属陶瓷材料，在红外波段表现出较高的吸收特性，同时在可见光波段保持较高的透过率。
厚度范围：
SiO₂层：[50, 200] nm
厚度范围的选择基于其光学干涉效应，以优化可见光透过率。
Ag层：[10, 50] nm
厚度范围的选择基于Ag在红外波段的高反射性能，同时避免过厚导致可见光透过率下降。
TiN层：[20, 100] nm
厚度范围的选择基于其红外吸收能力，同时尽量减少对可见光透过率的影响。
结构构成机理分析
1. 可见光高透过：
SiO₂层的主要功能是确保可见光波段的高透过率。由于其低折射率和低吸收系数，SiO₂能够在不显著衰减光强度的情况下让可见光通过。
Ag层的厚度较薄（<50 nm），在可见光波段表现为半透明，允许大部分可见光透过。
TiN层在可见光波段的吸收较低，不会显著影响透过率。
2. 红外高反射：
Ag层能够有效地反射红外光。这是因为金属在红外波段通常表现出高反射率，且Ag的光学性能特别适合用于反射红外辐射。
SiO₂层对红外光的透过影响较小，主要由Ag层决定反射特性。
3. 红外高吸收：
TiN层可以吸收部分红外光，从而减少红外光的透过。TiN作为一种金属陶瓷材料，在红外波段具有一定的吸收特性，这有助于增强整体结构的红外吸收能力。
Ag层的厚度设计为部分透射红外光至TiN层，进一步提高吸收效率。
光学特性预测
透过率：
紫外（5-80）
可见光波段（200-800 nm）：
主要由SiO₂层和薄Ag层决定，透过率预计可达到80%-95%。
红外波段（700-2500 nm）：
透过率会显著降低，主要是因为Ag层的高反射和TiN层的吸收作用，预计透过率低于5%。
反射率：
可见光波段（400-700 nm）：
反射率较低，大部分光透过。
红外波段（700-2500 nm）：
Ag层将导致反射率显著增加，预计可达80%-95%。
吸收率：
紫外（5-80）
可见光波段（200-800 nm）：
吸收率较低，主要由SiO₂层的低吸收特性决定。
红外波段（1200-3000 nm）：
吸收率会因TiN层的存在而增加，预计可达到10%-30%。
总结
该复合薄膜结构通过合理选择材料及其厚度范围，可以在可见光波段实现高透过率（80%-95%），同时在红外波段实现高反射率（80%-95%）和吸收率（10%-30%）。用户可以通过进一步的进化差分运算优化各层的具体厚度，以获得最佳性能。建议重点优化Ag层和TiN层的厚度，以平衡红外反射与吸收性能。
"""
json_content,total_tokens1=communication_format(qw_model_name='qwen-turbo',full_content=full_content)
print(json_content)
print(total_tokens1)

{"build_composites": ["SiO₂", "Ag", "TiN"], "set_thickness": [[50, 200], [10, 50], [20, 100]], "wl": [200, 2500, 2300]}
1304


输出586```{
  "build_composites": ["SiO2", "Ag", "SiO2", "Al2O3", "SiO2"],
  "set_thickness": [50, 20, 50, 30, 50],
  "wl": [400, 2500, 2100]
}```

In [17]:
from utils.CalculatedMaterials import CalculatedMaterials
import os
#设置默认路径，为避免每次都要输入路径
os.chdir(r'E:\PycharmProjects\AgentMaterial')

CM=CalculatedMaterials(build_composites=[  'SiO2','Ag', 'TiO2'],
                      set_thickness=[50, 20, 50],
                      wl=[200, 1200,1000 ])
img_path,zipped=CM.calculate_fit_data(number_polyfit=[3],method='interpolite_composites')
print(img_path,zipped)

TypeError: MaterialTool.composites_calculate_rt_tmm() takes from 2 to 4 positional arguments but 5 were given

<Figure size 1200x1800 with 0 Axes>

# 上图判断是否符合预期

In [17]:
from pathlib import Path
def evaluation_model(qw_model_name: str,full_content: str,path:str):
    client = OpenAI(
    # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
            api_key="sk-568bd13551dd42ae9c623bd04504ba02",
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    system ="""
    你是一个复合薄膜材料工程师助手，现在用户会构建一个初始的复合材料模型，对这个材料的光学性能有一定的性能期望，他会通过计算
    得到材料的光学性能参数在不同波长下的透过率，吸收率和反射率。你需要根据用户提供的计算数据判断，是否符合用户对材料的光学性能
    期望，如果基本满足条件你就在'evaluation'键的值填写True,不满足就填False。你可以在'advice'键的值中写你的建议，以json的
    格式输出内容。输出形式为{'evaluation'：类型：bool,'advice'：你的建议，类型：str}
    """
    file_object=client.files.create(file=Path(path),purpose="file-extract")
    completion = client.chat.completions.create(
        model=qw_model_name,
        messages=[
            {'role': 'system', 'content': f'{system}'},
            {'role': 'user', 'content': f'fileid://{file_object.id}'},
            {'role': 'user', 'content': f'用户预期为：{full_content}'},
        ],
        response_format={"type": "json_object"}
    )
    # 获取生成的 JSON 内容
    json_content = completion.choices[0].message.content
    # 获取 token 使用情况
    total_tokens = completion.usage.total_tokens if hasattr(completion, 'usage') else None

    return json_content, total_tokens

In [20]:
 json_content, total_tokens=evaluation_model(qw_model_name='qwen-turbo',full_content='可见光高透过，红外光高反射或吸收',path=r'E:\PycharmProjects\AgentMaterial\content\image_temp\DE_image\img.png')
print(json_content)
print(total_tokens)

{
    "evaluation": true,
    "advice": "材料的光学性能基本满足用户需求。在可见光区域具有较高的透过率，同时在红外区域表现出较高的反射率或吸收率。继续保持当前设计参数，并进一步优化以增强红外反射或吸收性能。"
}
266


In [ ]:
import time 
data=range(50)
for i in data:
    time.sleep(1)
    print(i, end='', flush=True)

01234567891011121314151617181920212223242526272829303132333435363738394041424344454647

In [2]:
import os


def generate_file_tree(folder_path, prefix=''):
    tree = []
    items = os.listdir(folder_path)
    items.sort()
    for index, item in enumerate(items):
        full_path = os.path.join(folder_path, item)
        is_last = index == len(items) - 1
        if os.path.isdir(full_path) and not item.startswith('.'):
            tree.append(prefix + ('└── ' if is_last else '├── ') + item + '/')
            new_prefix = prefix + ('    ' if is_last else '│   ')
            tree.extend(generate_file_tree(full_path, new_prefix))
        else:
            tree.append(prefix + ('└── ' if is_last else '├── ') + item)
    return tree


def save_file_tree_to_txt(folder_path, output_file):
    tree = generate_file_tree(folder_path)
    with open(output_file, 'w', encoding='utf-8') as file:
        for line in tree:
            file.write(line + '\n')



folder_path = r'E:\PycharmProjects\AgentMaterial'  # 可以修改为你想要生成文件树的文件夹路径
output_file = 'file_tree.txt'  # 可以修改为你想要保存的文件名
save_file_tree_to_txt(folder_path, output_file)
print(f"文件树已保存到 {output_file}")

文件树已保存到 file_tree.txt


In [2]:
import os
from pathlib import Path
from openai import OpenAI
import base64
import numpy as np
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
client = OpenAI(
    api_key="sk-568bd13551dd42ae9c623bd04504ba02",  # 如果您没有配置环境变量，请在此处替换您的API-KEY
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)
def vl_model(qw_model_name: str,path:str,zipped:dict):
    base64_image = encode_image(path)
    zdict = {'fit_num': 1, 'data': zipped}
    client = OpenAI(
    # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
            api_key="sk-568bd13551dd42ae9c623bd04504ba02",
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    system = """
        你是一个数据拟合的评论大师，我会提供一张拟合结果的图片和拟合误差指标，你需要根据它们来分析并评判拟合结果
         拟合误差指标：用户一般填入{'fit_num':1,'data':{'Ag': {'n_error': {'MSE': 0.010203918504604354, 'RMSE': np.float64(0.10101444700934789), 'MAE': 0.05930365626546965, 'R^2': 0.9705055119102391}, 'k_error': {'MSE': 0.010206575730128196, 'RMSE': np.float64(0.10102759885362117), 'MAE': 0.05986610414526992, 'R^2': 0.9988746610651629}}, 'SiO2': {'n_error': {'MSE': 0.00011581736666310157, 'RMSE': np.float64(0.010761847734617953), 'MAE': 0.007805012017237496, 'R^2': 0.9725129989062254}, 'k_error': {'MSE': 0.0, 'RMSE': np.float64(0.0), 'MAE': 0.0, 'R^2': 1.0}}, 'TiO2': {'n_error': {'MSE': 0.00016698613248391714, 'RMSE': np.float64(0.012922311421874848), 'MAE': 0.009970185066536194, 'R^2': 0.9803852902836107}, 'k_error': {'MSE': 0.0, 'RMSE': np.float64(0.0), 'MAE': 0.0, 'R^2': 1.0}}}}
         拟合误差指标解释：fit_num代表,第一次进行拟合反馈给你结果，
                数据包含了三种材料（Ag、SiO2、TiO2）。
                每种材料有两个属性：折射率（n）和消光系数（k）。
                每个属性有四个误差指标：MSE（均方误差）、RMSE（均方根误差）、MAE（平均绝对误差）、R²（决定系数）。
                MSE 和 RMSE：衡量预测值与真实值之间的平均误差大小，值越小越好。
                MAE：衡量预测值与真实值之间的平均绝对误差，值越小越好。
                R²：衡量模型对数据的拟合程度，值越接近1越好。
                (1) Ag（银）
                折射率（n）：
                MSE = 0.0102, RMSE = 0.101, MAE = 0.059, R² = 0.971
                R²<=0.98），说明模型拟合效果不达标，于散点原数据有较大差距。
                RMSE 和 MAE 的值相对较大，说明预测值与真实值之间存在一定的误差。
                消光系数（k）：
                MSE = 0.0102, RMSE = 0.101, MAE = 0.060, R² = 0.999
                R² 非常接近1，说明模型对消光系数的拟合效果非常好。
                RMSE 和 MAE 的值略高于折射率，但整体误差仍然较小。
                (2) SiO2（二氧化硅）
                折射率（n）：
                MSE = 0.00012, RMSE = 0.011, MAE = 0.008, R² = 0.973
                R² <=0.98，图像拟合曲线和散点贴合程度不够，需要重新拟合。
                拟合曲线为一次线性曲线不符合要求
                RMSE 和 MAE 的值小，说明预测误差更小。
                消光系数（k）：
                MSE = 0.0, RMSE = 0.0, MAE = 0.0, R² = 1.0
                所有误差指标均为0，R²为1，说明模型对消光系数的拟合是完美的。
                (3) TiO2（二氧化钛）
                折射率（n）：
                MSE = 0.00017, RMSE = 0.013, MAE = 0.010, R² = 0.980
                R² 较高，说明模型对折射率的拟合效果较好，R²= 0.980符合要求。
                RMSE 和 MAE 的值比SiO2略大，但整体误差仍然较小。
                消光系数（k）：
                MSE = 0.0, RMSE = 0.0, MAE = 0.0, R² = 1.0
                所有误差指标均为0，R²为1，说明模型对消光系数的拟合是完美的。
         要求：
          1.以json的格式输出你的数据,例如上例你就需要返回{'Ag':{'n':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':False 类型为bool},'k':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':Ture 类型为bool}},
          'TiO2':{'n':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':Ture 类型为bool},'k':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':True 类型为bool}},
          'SiO2':{'n':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':False 类型为bool},'k':{'analysis':你的分析,拟合效果评价，图像整体形状 类型为str,'evaluation':Ture 类型为bool}}}
          其中anlaysis的值为你对该材料的n或者k的拟合效果的分析，evaluaion为你最后的判断拟合的好为True,拟合的不好为False
          2.上传的依据一般以图片的分析为主要判断（重点看与原数据的贴合程度，有较高要求），图中散点为原数据点，线为拟合数据，以拟合误差指标为辅助，需要看每个材料的n或k的整条曲线的拟合效果，客观的做出判断，拟合完美才行，R²需要>=0.98。
          3.以中文回复用户
          4.如果拟合曲线(红色拟合曲线)是一次的线性曲线是不可行的，直接在'evaluation'为False
          
        """
    completion = client.chat.completions.create(
        model=qw_model_name,
             messages=[
            {
                "role": "system",
                "content": [{"type":"text","text": system}]},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        # 需要注意，传入Base64，图像格式（即image/{format}）需要与支持的图片列表中的Content Type保持一致。"f"是字符串格式化的方法。
                        # PNG图像：  f"data:image/png;base64,{base64_image}"
                        # JPEG图像： f"data:image/jpeg;base64,{base64_image}"
                        # WEBP图像： f"data:image/webp;base64,{base64_image}"
                        "image_url": {"url": f"data:image/png;base64,{base64_image}"}, 
                    },
                    {"type": "text", "text":f'拟合误差指标{zdict},结合图片进行评价'},
                ],
            }
        ],response_format={"type": "json_object"}
)
    # 获取生成的 JSON 内容
    json_content = completion.choices[0].message.content
    # 获取 token 使用情况 
    total_tokens = completion.usage.total_tokens if hasattr(completion, 'usage') else None

    return json_content, total_tokens

{
    "Au": {
        "n": {
            "analysis": "Au的折射率（n）拟合曲线整体呈现非线性趋势，与原数据点的贴合程度较好，但R²值为0.9597，略低于0.98的标准，且RMSE和MAE值相对较大，表明预测值与真实值之间存在一定的误差。",
            "evaluation": false
        },
        "k": {
            "analysis": "Au的消光系数（k）拟合曲线与原数据点的贴合程度非常好，R²值高达0.9986，接近1，RMSE和MAE值较小，表明模型对消光系数的拟合效果非常理想。",
            "evaluation": true
        }
    },
    "SiO2": {
        "n": {
            "analysis": "SiO2的折射率（n）拟合曲线整体呈现非线性下降趋势，与原数据点的贴合程度较好，R²值为0.9725，略低于0.98的标准，但RMSE和MAE值较小，表明预测误差较小。",
            "evaluation": false
        },
        "k": {
            "analysis": "SiO2的消光系数（k）拟合曲线与原数据点完全重合，所有误差指标均为0，R²值为1，表明模型对消光系数的拟合是完美的。",
            "evaluation": true
        }
    },
    "TiO2": {
        "n": {
            "analysis": "TiO2的折射率（n）拟合曲线整体呈现非线性下降趋势，与原数据点的贴合程度非常好，R²值为0.9804，符合标准，RMSE和MAE值较小，表明预测误差较小。",
            "evaluation": true
        },
        "k": {
            "analysis": "TiO2的消光系数（k）拟合曲线与原数据点完全重合，所有误差指标均为0，R²值为1，表明模型对消光系数的拟合是完美的。",
            "evaluation": true
        }
    }
}


In [4]:
zipped={'Au': {'n_error': {'MSE': 0.010204066221765893, 'RMSE': np.float64(0.10101517817519252), 'MAE': 0.07625268995196206, 'R^2': 0.9597247950014287}, 'k_error': {'MSE': 0.010204168551898606, 'RMSE': np.float64(0.10101568468262048), 'MAE': 0.07510467534106234, 'R^2': 0.998612598722075}}, 'SiO2': {'n_error': {'MSE': 0.00011581736666310157, 'RMSE': np.float64(0.010761847734617953), 'MAE': 0.007805012017237496, 'R^2': 0.9725129989062254}, 'k_error': {'MSE': 0.0, 'RMSE': np.float64(0.0), 'MAE': 0.0, 'R^2': 1.0}}, 'TiO2': {'n_error': {'MSE': 0.00016698613248391714, 'RMSE': np.float64(0.012922311421874848), 'MAE': 0.009970185066536194, 'R^2': 0.9803852902836107}, 'k_error': {'MSE': 0.0, 'RMSE': np.float64(0.0), 'MAE': 0.0, 'R^2': 1.0}}}
json_content, total_tokens=vl_model(qw_model_name='qwen-vl-max-latest',path=fr"E:\PycharmProjects\AgentMaterial\content\image_temp\fit_image\['Au', 'SiO2', 'TiO2'].png",zipped=zipped)
print(json_content, total_tokens)

{
    "Au": {
        "n": {"analysis": "从图表上看，折反射率(n)的数据点分布较为分散，并且拟合曲线未能很好地捕捉到这些波动。尽管R²达到了较高的数值（约0.96），但是由于实际数据的变化幅度较大以及拟合曲线无法准确反映这种变化趋势。",
              "evaluation": False},
        "k": {"analysis": "对于消光系数(k)，虽然所有误差指标都显示得很好(R²达到近似1并且其他各项也较低)，但由于其对应的图形中的直线性质并不适合描述该类非线性的物理现象，在没有进一步验证的情况下不能完全确认是否合适。",
              "evaluation": True}
    },
    "SiO2": {
        "n": {"analysis": "折射率(n)的拟合表现良好,R²约为0.97表明了较好的吻合度；同时可以看到残差很小(MAE极低)，这表示模型能够很好的逼近原始数据的趋势",
              "evaluation": True},
        "k": {"analysis": "消光系数(k)的情况则非常理想，所有的统计量都是零意味着没有任何偏差出现，因此可以认为这个模型非常适合用来预测此参数.",
              "evaluation": True}
    },
    "TiO2": {
        "n": {"analysis": "在这个例子中,TiO2的折射率(n)表现出色，具有很高的R²值大约等于0.98，这意味着它能很好地区分出不同条件下的行为模式;另外也可以看到它的MAE很低，证明我们的回归方法相当精确地再现出了实验测量的结果;",
              "evaluation": True},
        "k": {"analysis": "同样情况下,k的表现也非常优秀，因为这里的统计数据全部归零，所以我们可以确定我们所建立起来的关系式是非常有效的。",
              "evaluation": True}
    }
}
```

### 解析：

- **金(Au)**:
  - 对于**折射率**(n): 图像上的散点比较密集而弯曲复杂，聚合成一

In [2]:
x={
    "Au": {
        "n": {
            "analysis": "Au的折射率（n）拟合曲线整体呈现非线性趋势，与原数据点的贴合程度较好，但R²值为0.9597，略低于0.98的标准，且RMSE和MAE值相对较大，表明预测值与真实值之间存在一定的误差。",
            "evaluation": False
        },
        "k": {
            "analysis": "Au的消光系数（k）拟合曲线与原数据点的贴合程度非常好，R²值高达0.9986，接近1，RMSE和MAE值较小，表明模型对消光系数的拟合效果非常理想。",
            "evaluation": True
        }
    },
    "SiO2": {
        "n": {
            "analysis": "SiO2的折射率（n）拟合曲线整体呈现非线性下降趋势，与原数据点的贴合程度较好，R²值为0.9725，略低于0.98的标准，但RMSE和MAE值较小，表明预测误差较小。",
            "evaluation": False
        },
        "k": {
            "analysis": "SiO2的消光系数（k）拟合曲线与原数据点完全重合，所有误差指标均为0，R²值为1，表明模型对消光系数的拟合是完美的。",
            "evaluation": True
        }
    },
    "TiO2": {
        "n": {
            "analysis": "TiO2的折射率（n）拟合曲线整体呈现非线性下降趋势，与原数据点的贴合程度非常好，R²值为0.9804，符合标准，RMSE和MAE值较小，表明预测误差较小。",
            "evaluation": True
        },
        "k": {
            "analysis": "TiO2的消光系数（k）拟合曲线与原数据点完全重合，所有误差指标均为0，R²值为1，表明模型对消光系数的拟合是完美的。",
            "evaluation": True
        }
    }
}

In [6]:
for i in x:
    for j in x[i]:
        if not x[i][j]['evaluation']:
            print(i,j)

Au n
SiO2 n
